# List Generator

In section 09, we have seen list comprehension. List comprehensions are useful and can help you write elegant code that’s easy to read and debug, but they’re not the right choice for all circumstances.

For example, when you handle a big list, list comprehension may consume all your memory. Because **A list comprehension in Python works by loading the entire output list into memory**.

To solve this problem, we need to use a **generator** instead of a list comprehension. **A generator doesn’t create a single, large data structure in memory, but instead returns an iterable**.